In [1]:
import pandas as pd
import numpy as np
import QuantLib as ql

## Interest Rate Swaps in QuantLib

Plain Vanilla USD swap 10m notional 10Y maturity 

Fixed leg 3% cupon semianually, floating leg pays 3.5m quarterly

In [13]:
calculation_date = ql.Date(15,1,2012)
ql.Settings.instance().evaluationDate = calculation_date

In [14]:
risk_free_rate = 0.01
libor_rate = 0.02

day_count = ql.Actual365Fixed()

discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date,risk_free_rate,day_count))

libor_curve = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date,libor_rate,day_count))

libor3M_index = ql.USDLibor(ql.Period(3,ql.Months), libor_curve)

## To make the swap, construct the fixed and floating leg schedules

In [15]:
calendar = ql.UnitedStates()

settle_date = calendar.advance(calculation_date,5,ql.Days)
maturity_date = calendar.advance(settle_date,10,ql.Years)


fixed_leg_tenor = ql.Period(6,ql.Months)
fixed_schedule = ql.Schedule(settle_date, maturity_date, fixed_leg_tenor,calendar,ql.ModifiedFollowing, 
                             ql.ModifiedFollowing,ql.DateGeneration.Forward,False)

float_leg_tenor = ql.Period(3,ql.Months)
float_schedule = ql.Schedule(settle_date, maturity_date, fixed_leg_tenor,calendar,ql.ModifiedFollowing, 
                             ql.ModifiedFollowing,ql.DateGeneration.Forward,False)

In [16]:
notional = 1*10**6

fixed_rate  = 0.03
fixed_leg_daycount = ql.Actual360()
float_spread  = 0.004
float_leg_daycount = ql.Actual360()

ir_swap = ql.VanillaSwap(ql.VanillaSwap.Payer, notional, fixed_schedule, fixed_rate,fixed_leg_daycount,float_schedule, 
                         libor3M_index, float_spread,float_leg_daycount)

swap_engine = ql.DiscountingSwapEngine(discount_curve)
ir_swap.setPricingEngine(swap_engine)

In [17]:
for i, cf in enumerate(ir_swap.leg(0)):
    print("%2d    %-18s  %10.2f"%(i+1, cf.date(), cf.amount()))

 1    July 23rd, 2012       15166.67
 2    January 23rd, 2013    15333.33
 3    July 23rd, 2013       15083.33
 4    January 23rd, 2014    15333.33
 5    July 23rd, 2014       15083.33
 6    January 23rd, 2015    15333.33
 7    July 23rd, 2015       15083.33
 8    January 25th, 2016    15500.00
 9    July 25th, 2016       15166.67
10    January 23rd, 2017    15166.67
11    July 24th, 2017       15166.67
12    January 23rd, 2018    15250.00
13    July 23rd, 2018       15083.33
14    January 23rd, 2019    15333.33
15    July 23rd, 2019       15083.33
16    January 23rd, 2020    15333.33
17    July 23rd, 2020       15166.67
18    January 25th, 2021    15500.00
19    July 23rd, 2021       14916.67
20    January 24th, 2022    15416.67


In [20]:
print("%-20s: %20.3f" % ("Net Present Value", ir_swap.NPV()))
print("%-20s: %20.3f" % ("Fair Spread", ir_swap.fairSpread()))
print("%-20s: %20.3f" % ("Fair Rate", ir_swap.fairRate()))
print("%-20s: %20.3f" % ("Fixed Leg BPS", ir_swap.fixedLegBPS()))
print("%-20s: %20.3f" % ("Floating Leg BPS", ir_swap.floatingLegBPS()))

('Net Present Value', -59477.931577944895)
('Fair Spread', 0.010174896073070736)
Fair Rate           :                0.024
Fixed Leg BPS       :             -963.222
Floating Leg BPS    :              963.222
